In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from sklearn.metrics import classification_report


In [59]:
df = pd.read_csv("dataset_1.csv")

df.columns

Index(['Unnamed: 0', 'date', 'id', 'meancircumplex.arousal',
       'meancircumplex.valence', 'countappCat.builtin',
       'countappCat.communication', 'countappCat.entertainment',
       'countappCat.finance', 'countappCat.game', 'countappCat.office',
       'countappCat.other', 'countappCat.social', 'countappCat.travel',
       'countappCat.unknown', 'countappCat.utilities', 'countappCat.weather',
       'countcall', 'countcircumplex.arousal', 'countcircumplex.valence',
       'countscreen', 'countsms', 'sumactivity', 'sumappCat.builtin',
       'sumappCat.communication', 'sumappCat.entertainment',
       'sumappCat.finance', 'sumappCat.game', 'sumappCat.office',
       'sumappCat.other', 'sumappCat.social', 'sumappCat.travel',
       'sumappCat.unknown', 'sumappCat.utilities', 'sumappCat.weather',
       'sumcall', 'sumscreen', 'sumsms', 'time_window_meancircumplex.valence',
       'time_window_sumcircumplex.valence', 'time_window_countactivity',
       'time_window_countcircumplex

In [60]:
from sklearn.model_selection import train_test_split

# Splitting the data into train, validation, and test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# First, split into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(df[['meancircumplex.arousal',
       'meancircumplex.valence', 'countappCat.builtin',
       'countappCat.communication', 'countappCat.entertainment',
       'countappCat.finance', 'countappCat.game', 'countappCat.office',
       'countappCat.other', 'countappCat.social', 'countappCat.travel',
       'countappCat.unknown', 'countappCat.utilities', 'countappCat.weather',
       'countcall', 'countcircumplex.arousal', 'countcircumplex.valence',
       'countscreen', 'countsms', 'sumactivity', 'sumappCat.builtin',
       'sumappCat.communication', 'sumappCat.entertainment',
       'sumappCat.finance', 'sumappCat.game', 'sumappCat.office',
       'sumappCat.other', 'sumappCat.social', 'sumappCat.travel',
       'sumappCat.unknown', 'sumappCat.utilities', 'sumappCat.weather',
       'sumcall', 'sumscreen', 'sumsms', 'time_window_meancircumplex.valence',]], (df['bin']*0.1), test_size=test_ratio, random_state=42)

# Then, split train into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)

# Display the shape of each set
print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

Training set shape: (1274, 36)
Validation set shape: (274, 36)
Test set shape: (274, 36)


In [61]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.layers import Dropout


# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

input_size = X_train.shape[1]

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=20, verbose=1)



Epoch 1/100


In [ ]:

predictions = model.predict(X_test)
predictions = predictions*10
predictions = predictions.round()
y_test = y_test*10

# Print the predictions
print("Predictions:", predictions.flatten())

# Optionally, you can compare the predictions with the actual values
print("Actual values:", y_test)

report = classification_report(y_test, predictions)

print("Classification Report:")
print(report)

9/9 [==============================] - 0s 2ms/step
Predictions: [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 2. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 2. 0. 1. 1. 0.
 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1.
 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 2. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 2. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 2. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 1. 0. 0. 0. 1. 1. 1. 1. 1.]
Actual values: 555     0.0
1741    2.0
297     0.0
733     0.0
910     1.0
       ... 
1296    1.0
17

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into train, validation, and test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15



# First, split into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(df[['meancircumplex.arousal',
       'meancircumplex.valence', 'countappCat.builtin',
       'countappCat.communication', 'countappCat.entertainment',
       'countappCat.finance', 'countappCat.game', 'countappCat.office',
       'countappCat.other', 'countappCat.social', 'countappCat.travel',
       'countappCat.unknown', 'countappCat.utilities', 'countappCat.weather',
       'countcall', 'countcircumplex.arousal', 'countcircumplex.valence',
       'countscreen', 'countsms', 'sumactivity', 'sumappCat.builtin',
       'sumappCat.communication', 'sumappCat.entertainment',
       'sumappCat.finance', 'sumappCat.game', 'sumappCat.office',
       'sumappCat.other', 'sumappCat.social', 'sumappCat.travel',
       'sumappCat.unknown', 'sumappCat.utilities', 'sumappCat.weather',
       'sumcall', 'sumscreen', 'sumsms', 'time_window_meancircumplex.valence']], (df['bin']).round(0), test_size=test_ratio, random_state=42)

# Then, split train into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)

# Display the shape of each set
print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

Training set shape: (1274, 36)
Validation set shape: (274, 36)
Test set shape: (274, 36)


In [ ]:
print(df['handmatige_bin'].min())
print(df['handmatige_bin'].max())

0
2


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import metrics





# Define the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')  # Output layer with 10 units for 10 classes and softmax activation
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
y_pred = model.predict(X_test)

y_pred_new = []
for i in y_pred:
    max_val = max(i)
    for idx, j in enumerate(i):
        if j == max_val:
            y_pred_new.append(idx)

report = classification_report(y_test, y_pred_new)

print("Classification Report:")
print(report)


Epoch 1/100
36/36 [==============================] - 1s 9ms/step - loss: 0.9964 - accuracy: 0.6003 - val_loss: 0.8740 - val_accuracy: 0.7031
Epoch 2/100
36/36 [==============================] - 0s 4ms/step - loss: 0.8183 - accuracy: 0.6928 - val_loss: 0.7536 - val_accuracy: 0.7031
Epoch 3/100
36/36 [==============================] - 0s 4ms/step - loss: 0.7474 - accuracy: 0.7051 - val_loss: 0.7240 - val_accuracy: 0.7109
Epoch 4/100
36/36 [==============================] - 0s 4ms/step - loss: 0.7190 - accuracy: 0.7129 - val_loss: 0.7205 - val_accuracy: 0.7109
Epoch 5/100
36/36 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.7120 - val_loss: 0.7094 - val_accuracy: 0.7031
Epoch 6/100
36/36 [==============================] - 0s 4ms/step - loss: 0.7041 - accuracy: 0.7138 - val_loss: 0.6942 - val_accuracy: 0.7031
Epoch 7/100
36/36 [==============================] - 0s 4ms/step - loss: 0.6857 - accuracy: 0.7190 - val_loss: 0.6968 - val_accuracy: 0.6719
Epoch 8/100
3

In [ ]:
y_pred_new = []
for i in y_pred:
    max_val = max(i)
    for idx, j in enumerate(i):
        if j == max_val:
            y_pred_new.append(idx)

print(y_pred_new)

[0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 2, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 1, 1, 1, 0, 0, 1, 2, 0, 1, 2, 1, 2, 0, 1, 2, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 2, 2, 0, 1, 2, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 2, 2, 1, 1, 1, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 1, 2, 0, 0, 0, 1, 1, 2, 0, 0, 1, 2, 1, 0, 0, 1, 2, 0, 1, 1, 1, 1, 1, 0, 2, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 2, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 2, 0, 2, 1, 1, 1, 0, 1, 2, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]


In [ ]:
# Train a decision tree

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


# Define the decision tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)

# Train the decision tree classifier
tree_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = tree_clf.predict(X_test)


report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.77      0.76       118
           1       0.58      0.63      0.61       110
           2       0.44      0.35      0.39        46

    accuracy                           0.64       274
   macro avg       0.60      0.58      0.59       274
weighted avg       0.64      0.64      0.64       274



In [ ]:
# Train an SVM

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# Define the multiclass SVM classifier
svm_clf = SVC(kernel='poly', decision_function_shape='ovr')  # One-vs-Rest (OvR) strategy for multiclass classification

# Train the SVM classifier
svm_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = svm_clf.predict(X_test)


report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.81      0.83       118
           1       0.62      0.85      0.72       110
           2       0.70      0.15      0.25        46

    accuracy                           0.72       274
   macro avg       0.72      0.60      0.60       274
weighted avg       0.73      0.72      0.69       274

